 <h1 align=center><font size = 6.5>Capstone Project</font></h1>

 <h1 align=center><font size = 6.5>Toronto vs New York- Battle of Neighborhoods</font></h1>

## <center> By- Pranish Naoghare

## Introduction

Toronto and New York are the famous places in the world. They are diverse in many ways. Both are multicultural as well as the financial hubs of their respective countries. We want to explore how much they are similar or dissimilar in aspects from a tourist point of view regarding food, accommodation, beautiful places, and many more.

Today Tourism is one of the pillars of the economy and the people most often visits those countries who are rich in heritage and developed enough from a foreign perspective, like friendly environment. Every city is unique in its own way and give something new. And now the information is so common regarding location of every place around the world on your fingertips which make it easier to explore. Therefore, tourists always eager to travel to different places on the basis of available information, and the comparison (the part of the information) between the two cities always assist to choose the specific places or according to their choice.

## Data

For this problem, we will get the services of Foursquare API to explore the data of two cities, in terms of their neighborhoods. The data also include the information about the places around each neighborhood like restaurants, hotels, coffee shops, parks, theaters, art galleries, museums and many more. We selected one Borough from each city to analyze their neighborhoods. Manhattan from New York and Downtown Toronto from Toronto. We will use machine learning technique, “Clustering” to segment the neighborhoods with similar objects on the basis of each neighborhood data. These objects will be given priority on the basis of foot traffic (activity) in their respective neighborhoods. This will help to locate the tourist’s areas and hubs, and then we can judge the similarity or dissimilarity between two cities on that basis.

## Methodology

Here both the city Boroughs, i.e. Downtown Toronto and Manhatten, will be analyzed. These cities will be subjected to data exploration , analysis and visualization seperately. 

For Downtown Toronto the neighborhood datais not readily available on the internet. But there exists a Wikipedia Page exists that has all the information that we need to explore and cluster the neighboroods in Toronto. This data will have to be scraped from the page, wrangle and clean it and then read into the pandas datframe. Cleaing involved droping the rows to which Borough was "Not Assigned". ALso, later on coordinates of each borough were obtained from the internet and then merged into the previous dataframe.

For Manhattan, a saved data file was explored through Foursquare Api. In this all boroughs along with their cordinates are readily available.

Both the neighborhoods were then explored seperately as veues and venue categories and then they were subjected to machine learning model of "Clustering" using "k-means" technique.

__Importing Basic Libraries__

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

__Import urllib and BeautifulSoup library__

* We need to import the __urllib__ library to connect to the Wikipedia page and fetch the contents of that page.
* We want to import the functions from __Beautiful Soup__ which will let us parse and work with the HTML we fetched from our Wiki page.

In [3]:
# import the library we use to open URLs
import urllib.request
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup

## 1. Downloading and Loading Data

### A. Scraping Data for Toronto from Wikipedia Site

In [4]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

In [5]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")
type(soup)

bs4.BeautifulSoup

In [6]:
# Get the title
title = soup.title
print(title)

<title>List of postal codes of Canada: M - Wikipedia</title>


We send Beautiful Soup off to retrieve all instances of the _table_ tag within the page and add them to an array called __all_tables__

In [7]:
all_tables=soup.find_all("table")

In [8]:
right_table=soup.find('table', class_='wikitable sortable')

Start looping through the rows to get the data for every Postal Code in the table.

There are three columns in our table that we want to scrape the data from so we will set up three empty lists (A, B and C) to store our data in.

We want to use the Beautiful Soup ‘find_all’ function again and set it to look for the string ‘tr’. We will then set up a FOR loop for each row within that array and set Python to loop through the rows, one by one.

Within the loop we are going to use find_all again to search each row for _td_ tags with the ‘td’ string. We will add all of these to a variable called ‘cells’ and then check to make sure that there are 3 items in our ‘cells’ array (i.e. one for each column).

If there are then we use the find(text=True)) option to extract the content string from within each _td_ element in that row and add them to the A-C lists we created at the start of this step. Let’s have a look at the code:

In [9]:
A=[]
B=[]
C=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True)[:-1])
        B.append(cells[1].find(text=True)[:-1])
        C.append(cells[2].find(text=True)[:-1])


We wil create a dataframe with it, assigning each of the lists A-C into a column with the name of our source table columns

In [10]:
import pandas as pd
tor_neigh=pd.DataFrame(A,columns=['PostalCode'])
tor_neigh['Borough']=B
tor_neigh['Neighborhood']=C
tor_neigh.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [11]:
tor_neigh.shape[0]

180

Next, we will be __dropping__ all the rows from the dataframe which are __'Not assigned'__ under the column __Borough__

In [12]:
for i in range(tor_neigh.shape[0]):
    if (tor_neigh.loc[i,'Borough']=='Not assigned'):
        tor_neigh.drop(index=i,inplace=True)

In [13]:
nrows=tor_neigh.shape[0]
print('Number of rows in the dataframe=',nrows)

Number of rows in the dataframe= 103


In [14]:
tor_neigh=tor_neigh.reset_index().drop(columns=['index'])

Grouping the Neighborhoods for same Postal Code.

In [15]:
tor_neigh = tor_neigh.groupby('PostalCode').agg({'Borough':'first', 
                             'Neighborhood': ', '.join}).reset_index()

In [16]:
tor_neigh.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Lets find the number of rows in the dataframe.

In [17]:
nrows=tor_neigh.shape[0]
print('Number of rows in the dataframe=',nrows)

Number of rows in the dataframe= 103


In [18]:
df_geo=pd.read_csv('Geospatial_Coordinates.csv')

In [19]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
tor_neigh=tor_neigh.join(df_geo.set_index('Postal Code'), on='PostalCode')

Quickly examine the resulting dataframe.

In [21]:
tor_neigh.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [22]:
tor_neigh.columns

Index(['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

In [23]:
# eliminate 'Postcode' column
tor_neigh=tor_neigh.drop(['PostalCode'],axis=1)
tor_neigh.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


Let's check how many Boroughs and Neighborhoods are present in the data.

In [24]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(tor_neigh['Borough'].unique()),
        tor_neigh.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


Let's slice the original dataframe and create a new dataframe of the Toronto data, segment and cluster only the neighborhoods in Toronto. 

In [25]:
toronto_data=tor_neigh[tor_neigh['Borough']=='Downtown Toronto'].reset_index(drop=True)
toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,Rosedale,43.679563,-79.377529
1,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


### B. Downloading Data for New York through a .json link

In [26]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


'wget' is not recognized as an internal or external command,
operable program or batch file.


#### Load and explore the data

Next, let's load the data.

In [27]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [28]:
neighborhoods_data = newyork_data['features']

#### Tranform the data into a *pandas* dataframe

In [29]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
ny_neigh = pd.DataFrame(columns=column_names)

In [30]:
for data in neighborhoods_data:
    borough =  data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    ny_neigh = ny_neigh.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [31]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(ny_neigh['Borough'].unique()),
        ny_neigh.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


Let's slice the original dataframe and create a new dataframe of the Manhattan, NY data, segment and cluster only the neighborhoods in Manhattan, NY. 

In [32]:
manhattan_data = ny_neigh[ny_neigh['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


## 2. Visualization

### A. Use geopy library to get the latitude and longitude values of Downtown Toronto.

Let's get the geographical coordinates of Downtown Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [75]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
tor_location = geolocator.geocode(address)
tor_latitude = tor_location.latitude
tor_longitude = tor_location.longitude
print('The geograpical coordinate of Downtown Toronto City are {}, {}.'.format(tor_latitude, tor_longitude))

The geograpical coordinate of Downtown Toronto City are 43.6534817, -79.3839347.


Let's visualize Downtown Toronto the neighborhoods in it.

In [34]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[tor_latitude, tor_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### B. Use geopy library to get the latitude and longitude values of Manhattan.

Let's get the geographical coordinates of Manhattan.

In [35]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
man_location = geolocator.geocode(address)
man_latitude = man_location.latitude
man_longitude = man_location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(man_latitude, man_longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [36]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[man_latitude, man_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [37]:
CLIENT_ID = '2GIKA0KP5L0HZ2WLV2JKAHOFFN2PWSJ1TWLHP4H31JOFWFE1' # your Foursquare ID
CLIENT_SECRET = 'Y1R12IF0EK1F2VQMVPXYNWP4LMWXD4L0DCB3YMEJZCLRNWQQ' # your Foursquare Secret
VERSION = '20180604'
limit=20
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2GIKA0KP5L0HZ2WLV2JKAHOFFN2PWSJ1TWLHP4H31JOFWFE1
CLIENT_SECRET:Y1R12IF0EK1F2VQMVPXYNWP4LMWXD4L0DCB3YMEJZCLRNWQQ


## 3. Analysis 

## A. Explore Neighborhoods in Downtown Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Downtown Toronto

In [38]:
# Let's create a function to repeat the process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<a id='item2'></a>

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [39]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


#### Let's check the size of the resulting dataframe

In [40]:
print(toronto_venues.shape)
toronto_venues.head()

(356, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


Let's check how many venues were returned for each neighborhood

In [41]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,20,20,20,20,20,20
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,20,20,20,20,20,20
Christie,16,16,16,16,16,16
Church and Wellesley,20,20,20,20,20,20
"Commerce Court, Victoria Hotel",20,20,20,20,20,20
"First Canadian Place, Underground city",20,20,20,20,20,20
"Garden District, Ryerson",20,20,20,20,20,20
"Harbourfront East, Union Station, Toronto Islands",20,20,20,20,20,20


#### Let's find out how many unique categories can be curated from all the returned venues

In [42]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 125 uniques categories.


## Analyze Each Neighborhood in Downtown Toronto

In [43]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [44]:
toronto_onehot.shape

(356, 125)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [45]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

#### Let's confirm the new size

In [46]:
toronto_grouped.shape

(19, 125)

#### Let's print each neighborhood along with the top 5 most common venues

In [47]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0  Seafood Restaurant  0.10
1        Cocktail Bar  0.05
2  Basketball Stadium  0.05
3         Coffee Shop  0.05
4          Restaurant  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0     Airport Service  0.19
1      Airport Lounge  0.12
2    Airport Terminal  0.12
3     Harbor / Marina  0.06
4  Airport Food Court  0.06


----Central Bay Street----
                        venue  freq
0                 Coffee Shop  0.30
1   Middle Eastern Restaurant  0.05
2                        Park  0.05
3                         Spa  0.05
4  Modern European Restaurant  0.05


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3     Baby Store  0.06
4    Coffee Shop  0.06


----Church and Wellesley----
             venue  freq
0      Pizza Place  0.05
1  Bubble Tea Shop  0.05
2     Burger Joint 

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
tor_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
tor_neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    tor_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

tor_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Basketball Stadium,Bakery,Park,Cocktail Bar,Coffee Shop,Museum,Breakfast Spot,Concert Hall,Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Boat or Ferry,Boutique,Rental Car Location,Sculpture Garden,Airport Gate
2,Central Bay Street,Coffee Shop,Café,Sushi Restaurant,Modern European Restaurant,Bubble Tea Shop,Ramen Restaurant,Middle Eastern Restaurant,Sandwich Place,Spa,Japanese Restaurant
3,Christie,Grocery Store,Café,Park,Candy Store,Diner,Coffee Shop,Nightclub,Restaurant,Italian Restaurant,Baby Store
4,Church and Wellesley,Pizza Place,Dance Studio,Park,Coffee Shop,Mexican Restaurant,Burger Joint,Bubble Tea Shop,Breakfast Spot,Ramen Restaurant,Martial Arts Dojo


## Clustering Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [50]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 4, 0, 3, 3, 3, 0, 3])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [51]:
# add clustering labels
tor_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(tor_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Deli / Bodega,Cheese Shop,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym
1,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,3,Restaurant,Café,Gift Shop,Indian Restaurant,Deli / Bodega,Jewelry Store,Diner,Bakery,Japanese Restaurant,Italian Restaurant
2,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Pizza Place,Dance Studio,Park,Coffee Shop,Mexican Restaurant,Burger Joint,Bubble Tea Shop,Breakfast Spot,Ramen Restaurant,Martial Arts Dojo
3,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Breakfast Spot,Farmers Market,Chocolate Shop,Pub,Restaurant,Performing Arts Venue,Dessert Shop,Bakery
4,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Café,Mexican Restaurant,Shopping Mall,Restaurant,Ramen Restaurant,Coffee Shop,Plaza,Steakhouse,Art Gallery,Sandwich Place


Finally, let's visualize the resulting clusters

In [52]:
# create map
map_clusters = folium.Map(location=[tor_latitude, tor_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1- Airport Lounge and Service, Coffee Shop, Cafe, Restaurants & Grocery Store

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Church and Wellesley,Pizza Place,Dance Studio,Park,Coffee Shop,Mexican Restaurant,Burger Joint,Bubble Tea Shop,Breakfast Spot,Ramen Restaurant,Martial Arts Dojo
6,Berczy Park,Seafood Restaurant,Basketball Stadium,Bakery,Park,Cocktail Bar,Coffee Shop,Museum,Breakfast Spot,Concert Hall,Restaurant
8,"Richmond, Adelaide, King",Coffee Shop,Seafood Restaurant,Gym / Fitness Center,Steakhouse,Hotel,Concert Hall,Lounge,Opera House,Café,Pizza Place
9,"Harbourfront East, Union Station, Toronto Islands",Park,Plaza,Hotel,Café,Supermarket,Performing Arts Venue,New American Restaurant,Bubble Tea Shop,Salad Place,Lake
12,"University of Toronto, Harbord",Restaurant,Bookstore,Bakery,Japanese Restaurant,Yoga Studio,Italian Restaurant,Café,College Gym,Comfort Food Restaurant,Sandwich Place
14,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Boat or Ferry,Boutique,Rental Car Location,Sculpture Garden,Airport Gate
15,Stn A PO Boxes,Café,Cocktail Bar,Tailor Shop,Museum,Comfort Food Restaurant,Restaurant,Concert Hall,Beer Bar,Park,Farmers Market


#### Cluster 2- Children Friendly- Parks, Playgrounds

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rosedale,Park,Playground,Trail,Deli / Bodega,Cheese Shop,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym


#### Cluster 3- Coffee Shops

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"Regent Park, Harbourfront",Coffee Shop,Park,Breakfast Spot,Farmers Market,Chocolate Shop,Pub,Restaurant,Performing Arts Venue,Dessert Shop,Bakery
7,Central Bay Street,Coffee Shop,Café,Sushi Restaurant,Modern European Restaurant,Bubble Tea Shop,Ramen Restaurant,Middle Eastern Restaurant,Sandwich Place,Spa,Japanese Restaurant
18,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Wings Joint,Park,Arts & Crafts Store,Beer Bar,Burrito Place,Creperie,Diner,Distribution Center


#### Cluster 4- Cafes and Restaurants

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"St. James Town, Cabbagetown",Restaurant,Café,Gift Shop,Indian Restaurant,Deli / Bodega,Jewelry Store,Diner,Bakery,Japanese Restaurant,Italian Restaurant
4,"Garden District, Ryerson",Café,Mexican Restaurant,Shopping Mall,Restaurant,Ramen Restaurant,Coffee Shop,Plaza,Steakhouse,Art Gallery,Sandwich Place
5,St. James Town,Gastropub,Coffee Shop,Café,Creperie,Art Gallery,Italian Restaurant,BBQ Joint,Cosmetics Shop,Food Truck,Restaurant
10,"Toronto Dominion Centre, Design Exchange",Café,Coffee Shop,Gym / Fitness Center,Steakhouse,Pub,Restaurant,Bookstore,Deli / Bodega,Beer Bar,Japanese Restaurant
11,"Commerce Court, Victoria Hotel",Café,Coffee Shop,Bakery,Gym,Gastropub,Ice Cream Shop,Japanese Restaurant,Deli / Bodega,Museum,Pub
13,"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Mexican Restaurant,Caribbean Restaurant,Bakery,Wine Bar,Fish Market,Farmers Market,Dessert Shop,Coffee Shop
16,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Gym / Fitness Center,Bookstore,Deli / Bodega,Bakery,Steakhouse,Pub,Seafood Restaurant


#### Cluster 5- Residential, Cafe

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Christie,Grocery Store,Café,Park,Candy Store,Diner,Coffee Shop,Nightclub,Restaurant,Italian Restaurant,Baby Store


## B. Explore Neighborhoods in Manhattan

In [58]:
# Let's create a function to repeat the process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues

In [59]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude'],
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


#### Let's check how many venues were returned for each neighborhood

In [60]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,20,20,20,20,20,20
Carnegie Hill,20,20,20,20,20,20
Central Harlem,20,20,20,20,20,20
Chelsea,20,20,20,20,20,20
Chinatown,20,20,20,20,20,20
Civic Center,20,20,20,20,20,20
Clinton,20,20,20,20,20,20
East Harlem,20,20,20,20,20,20
East Village,20,20,20,20,20,20


#### Let's find out how many unique categories can be curated from all the returned venues


In [61]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 213 uniques categories.


### Analyze Each Neighborhood in Manhattan

In [62]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
# Set Index
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

#### Let's print each neighborhood along with the top 5 most common venues

In [64]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0  Memorial Site  0.15
1           Park  0.15
2     Food Court  0.10
3       Building  0.05
4            Gym  0.05


----Carnegie Hill----
                  venue  freq
0  Gym / Fitness Center  0.10
1    Italian Restaurant  0.10
2                   Gym  0.10
3          Dance Studio  0.05
4          Gourmet Shop  0.05


----Central Harlem----
                venue  freq
0  African Restaurant  0.10
1   French Restaurant  0.10
2           Juice Bar  0.05
3          Bagel Shop  0.05
4                Café  0.05


----Chelsea----
                venue  freq
0  Seafood Restaurant  0.10
1      Ice Cream Shop  0.10
2             Theater  0.10
3            Beer Bar  0.05
4              Market  0.05


----Chinatown----
                 venue  freq
0       Sandwich Place  0.10
1   Chinese Restaurant  0.10
2  Indie Movie Theater  0.05
3             Tea Room  0.05
4          Pizza Place  0.05


----Civic Center----
                             venue  

#### Let's put that into a pandas dataframe

In [65]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [66]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
man_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
man_neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    man_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

man_neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Memorial Site,Park,Food Court,Cooking School,Sandwich Place,Shopping Mall,Smoke Shop,Gym,BBQ Joint,Auditorium
1,Carnegie Hill,Gym,Italian Restaurant,Gym / Fitness Center,Community Center,Spa,Café,Shoe Store,Bookstore,Gourmet Shop,Coffee Shop
2,Central Harlem,French Restaurant,African Restaurant,Dessert Shop,Bagel Shop,Ethiopian Restaurant,Boutique,Library,Gym / Fitness Center,Beer Bar,Café
3,Chelsea,Theater,Ice Cream Shop,Seafood Restaurant,Scenic Lookout,Italian Restaurant,Chinese Restaurant,Market,Office,Beer Bar,Hotel
4,Chinatown,Sandwich Place,Chinese Restaurant,Indie Movie Theater,Tea Room,Hotpot Restaurant,Ice Cream Shop,Greek Restaurant,Museum,English Restaurant,New American Restaurant
5,Civic Center,Spa,French Restaurant,Yoga Studio,Gym,Monument / Landmark,Molecular Gastronomy Restaurant,Falafel Restaurant,Martial Arts Dojo,Furniture / Home Store,Sushi Restaurant
6,Clinton,Theater,Gym / Fitness Center,Hotel,Sporting Goods Shop,Peruvian Restaurant,Comedy Club,Cocktail Bar,Café,Building,Food Court
7,East Harlem,Mexican Restaurant,Latin American Restaurant,Thai Restaurant,Sandwich Place,Park,Beer Bar,Street Art,Steakhouse,Bakery,Cocktail Bar
8,East Village,Vietnamese Restaurant,Dessert Shop,Cheese Shop,Japanese Restaurant,Beer Store,Swiss Restaurant,Coffee Shop,Juice Bar,Bar,Dog Run
9,Financial District,Coffee Shop,Gym / Fitness Center,Jewelry Store,Falafel Restaurant,Café,Steakhouse,New American Restaurant,Seafood Restaurant,Salad Place,Spa


## Clustering Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [67]:
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 1, 0, 1, 0, 4, 0, 0, 2])

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [68]:
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(man_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Gym,Coffee Shop,Yoga Studio,Diner,Pharmacy,Seafood Restaurant,Steakhouse,Supplement Shop,Sandwich Place,Donut Shop
1,Manhattan,Chinatown,40.715618,-73.994279,2,Sandwich Place,Chinese Restaurant,Indie Movie Theater,Tea Room,Hotpot Restaurant,Ice Cream Shop,Greek Restaurant,Museum,English Restaurant,New American Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Wine Shop,Café,Park,Deli / Bodega,Ramen Restaurant,Breakfast Spot,Market,Liquor Store,Cocktail Bar,Coffee Shop
3,Manhattan,Inwood,40.867684,-73.921210,0,Wine Bar,Café,Park,Yoga Studio,Diner,Spanish Restaurant,Farmers Market,Frozen Yogurt Shop,Bistro,Latin American Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Yoga Studio,Mexican Restaurant,Cocktail Bar,Japanese Restaurant,Café,Smoke Shop,Caribbean Restaurant,Mediterranean Restaurant,Coffee Shop,Bakery


In [69]:
map_clusters = folium.Map(location=[man_latitude, man_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


## Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1-  Residential

In [70]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Gym,Coffee Shop,Yoga Studio,Diner,Pharmacy,Seafood Restaurant,Steakhouse,Supplement Shop,Sandwich Place,Donut Shop
3,Inwood,Wine Bar,Café,Park,Yoga Studio,Diner,Spanish Restaurant,Farmers Market,Frozen Yogurt Shop,Bistro,Latin American Restaurant
5,Manhattanville,Italian Restaurant,BBQ Joint,Gastropub,Sushi Restaurant,Bike Trail,Coffee Shop,Lounge,Café,Juice Bar,Dumpling Restaurant
7,East Harlem,Mexican Restaurant,Latin American Restaurant,Thai Restaurant,Sandwich Place,Park,Beer Bar,Street Art,Steakhouse,Bakery,Cocktail Bar
8,Upper East Side,Hotel,Cosmetics Shop,Bar,Italian Restaurant,Jazz Club,Gym / Fitness Center,French Restaurant,Optical Shop,Park,Pet Store
10,Lenox Hill,Thai Restaurant,Gym,Cycle Studio,Liquor Store,Chinese Restaurant,Taco Place,Salad Place,French Restaurant,Restaurant,College Academic Building
11,Roosevelt Island,Coffee Shop,Residential Building (Apartment / Condo),Bus Line,Farmers Market,Food & Drink Shop,School,Dog Run,Liquor Store,Sandwich Place,Outdoors & Recreation
15,Midtown,Clothing Store,Hotel,Miscellaneous Shop,French Restaurant,Plaza,Park,Salad Place,Smoke Shop,Spa,Cycle Studio
20,Lower East Side,Art Gallery,Cocktail Bar,Yoga Studio,Japanese Restaurant,Bubble Tea Shop,French Restaurant,Filipino Restaurant,Chinese Restaurant,Mediterranean Restaurant,Coffee Shop
21,Tribeca,Park,Yoga Studio,Italian Restaurant,Playground,Poke Place,Coffee Shop,Salad Place,Café,Men's Store,Spa


#### Cluster 2- Restaurants and cafes

In [71]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Wine Shop,Café,Park,Deli / Bodega,Ramen Restaurant,Breakfast Spot,Market,Liquor Store,Cocktail Bar,Coffee Shop
4,Hamilton Heights,Yoga Studio,Mexican Restaurant,Cocktail Bar,Japanese Restaurant,Café,Smoke Shop,Caribbean Restaurant,Mediterranean Restaurant,Coffee Shop,Bakery
13,Lincoln Square,Theater,Indie Movie Theater,Concert Hall,Performing Arts Venue,College Arts Building,Circus,Opera House,Fountain,Gift Shop,Library
18,Greenwich Village,Italian Restaurant,Yoga Studio,Sushi Restaurant,Beer Bar,Clothing Store,New American Restaurant,Gourmet Shop,Coffee Shop,French Restaurant,Jazz Club
19,East Village,Vietnamese Restaurant,Dessert Shop,Cheese Shop,Japanese Restaurant,Beer Store,Swiss Restaurant,Coffee Shop,Juice Bar,Bar,Dog Run
38,Flatiron,Cycle Studio,Wine Shop,American Restaurant,Japanese Restaurant,Gym / Fitness Center,Café,Sports Club,Furniture / Home Store,Sushi Restaurant,Donut Shop


#### Cluster 3- Going Out/ Entertainment

In [72]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Sandwich Place,Chinese Restaurant,Indie Movie Theater,Tea Room,Hotpot Restaurant,Ice Cream Shop,Greek Restaurant,Museum,English Restaurant,New American Restaurant
9,Yorkville,Wine Shop,Deli / Bodega,Gym,Dog Run,Café,Liquor Store,Sandwich Place,Beer Store,Coffee Shop,Gym / Fitness Center
12,Upper West Side,Bakery,American Restaurant,Italian Restaurant,Pub,Bagel Shop,Bookstore,Tiki Bar,Chinese Restaurant,Greek Restaurant,Movie Theater
16,Murray Hill,Coffee Shop,Hotel,Burger Joint,Jewish Restaurant,Sandwich Place,Tea Room,Bar,Grocery Store,Event Space,Bagel Shop
22,Little Italy,Mediterranean Restaurant,Bakery,History Museum,Thai Restaurant,Mexican Restaurant,Spanish Restaurant,French Restaurant,Chinese Restaurant,Sandwich Place,Chocolate Shop
27,Gramercy,Pizza Place,Coffee Shop,Italian Restaurant,Yoga Studio,Bagel Shop,Sushi Restaurant,Gourmet Shop,Beer Bar,Bar,Park
31,Noho,Sandwich Place,Rock Club,Wine Shop,Italian Restaurant,Coffee Shop,French Restaurant,Boutique,Gourmet Shop,Greek Restaurant,Grocery Store
36,Tudor City,Park,Yoga Studio,Boxing Gym,Sushi Restaurant,Taco Place,Café,Seafood Restaurant,Thai Restaurant,Japanese Restaurant,Spanish Restaurant
39,Hudson Yards,American Restaurant,Hotel,Gym / Fitness Center,Park,Furniture / Home Store,Supermarket,Cocktail Bar,Building,Theater,Pedestrian Plaza


#### Cluster 4- Restaurant, Cafe, Entertainment

In [73]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Chelsea,Theater,Ice Cream Shop,Seafood Restaurant,Scenic Lookout,Italian Restaurant,Chinese Restaurant,Market,Office,Beer Bar,Hotel


#### Cluster 5- Restaurants, cafes, Residential and Music Venues

In [74]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Harlem,French Restaurant,African Restaurant,Dessert Shop,Bagel Shop,Ethiopian Restaurant,Boutique,Library,Gym / Fitness Center,Beer Bar,Café
14,Clinton,Theater,Gym / Fitness Center,Hotel,Sporting Goods Shop,Peruvian Restaurant,Comedy Club,Cocktail Bar,Café,Building,Food Court


## Results

After clustering the data of the respective neighborhoods, both cities (Boroughs) have venues which can be explored and attract the Tourists. The neighborhoods are much similar in features like Theaters, opera houses, food places, clubs, museums, parks etc. As far as concern to dissimilarity, it differs in terms of some unique places like veterinary hospitals, historical places and monuments.

## Discussion and Recommendation

When I compared the tourist places, I observed that the historical place is only situated in Downtown Toronto and the Monument or landmark venue is in Manhattan neighborhoods. Similarly, Airport facility, Harbor, Sculpture garden and Boat or ferry services are also available in Downtown Toronto while venues like Nightlife, Climbing gym and Museums are present in Manhattan.

As far as concern to recommendations, I recommend Downtown Toronto Neighborhoods will be considered first to visit. The tourists have an easily travelling access due to Airport facility, which not only saves time but also helps to save money. This saved money can be utilized to explore more, the attracting venues.

## Conclusion

The Downtown Toronto and Manhattan neighborhoods have more like similar venues. As we know that every place is unique in its own way, so that’s argument is present in both neighborhoods. The dissimilarity exists in terms of some different venues and facilities but not on a larger extent